Dataset: https://www.kaggle.com/datasets/osmi/mental-health-in-tech-2016

In [220]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


%matplotlib inline

pd.options.display.max_columns = 63

df = pd.read_csv('data/mental.csv')

In [221]:
new_column_names = ['is_self_employed', 'employee_count_bracket', 'is_tech_org', 'role_is_IT', 'empl_provides_mh_cov', 'knows_mh_cov_options',
                    'empl_discussed_mh','empl_offers_mh_rsrcs','anon_protec','if_askfor_mh_medical_leave_how_easy', 'discuss_mh_with_empl_wouldcause_neg_conseq', 'discuss_ph_with_empl_wouldcause_neg_conseq',
                    'comfy_discussing_mh_with_coworkers','comfy_discussing_mh_with_supervisors' , 'empl_takes_mh_asseriously_as_ph', 'observed_neg_conseq_for_coworkers_openabout_mh_inworkspace',
                    'has_medical_cov_incl_mh', 'knowsof_mh_resources','if_diag_would_reveal_toclients/bn_contacts', 'if_reveal_diag_toclient_didthis_impact_neg', 'if_diag_would_reveal_tocoworkers/employees',
                    'if_reveal_diag_tocoworker_didthis_impact_neg', 'productivity_isaffected_by_mh', 'percentage_worktime_affected_by_mh', 'has_prev_employers','prev_empl_provided_mh_benefits',
                    'was_aware_of_prevemployers_mhcare_options','prev_empl_discussed_mh', 'prev_empl_provided_mh_rsrc', 'prev_anon_protec', 'prev_discuss_mh_with_empl_wouldcause_neg_conseq',
                    'prev_discuss_ph_with_empl_wouldcause_neg_conseq', 'prev_wouldhavebeen_willing_discuss_mh_coworkers', 'prev_wouldhavebeen_willing_discuss_mh_supervisors',
                    'prev_empl_takes_mh_asseriously_as_ph', 'prev_observed_neg_conseq_for_coworkers_openabout_mh_inworkspace', 'willingto_bringup_ph_interview', 'why', 'willingto_bringup_mh_interview',
                    'why2', 'believes_beingident_as_mh_wouldhurt_career', 'thinks_coworkers_wouldviewthem_neg_if_mh', 'howwilling_share_mh_with_friendsfamily',
                    'observed_badly_handled_response_to_mh_inworkplace', 'observed_madethem_notwantto_talk_about_mh', 'has_family_history_mh', 'hashad_mh_inpast', 'HAS_MH', 'diagnoses_notprof',
                    'conditions_suspected', 'hasbeen_diag_byprof', 'diagnoses_prof', 'sought_treatment', 'mh_interferes_w/work_effective_trt', 'mh_interferes_w/work_ineffective_trt',
                    'age', 'sex', 'country_livesin', 'us_state_livesin', 'country_worksin', 'us_state_worksin', 'work_position', 'is_remote_working']

df.columns = new_column_names

In [222]:
why_cols = [columns for columns in df.columns if 'why' in columns]

#dropping 'why' questions, as each one of them is too unique
for column in why_cols:
    df.drop(column, axis='columns', inplace=True)

#most people (all but 26) work in the same country as they live. this info will still be preserved in remote working column. dropping the country_livesin_column
df.drop('country_livesin', axis='columns', inplace=True)

In [223]:
#dropping the columns that most rows are missing values in
#columns_too_many_missing = [column for column in df.columns if df[column].isna().sum() > 0.5 * df.shape[0]]
#df = df.drop(columns_too_many_missing, axis='columns')

In [224]:
df.drop(['us_state_livesin', 'us_state_worksin'], axis='columns', inplace=True)

In [225]:
#dropping rows with 25 missing values or more
df = df.loc[df.isna().sum(axis=1) < 25]

In [226]:
#Replacing gender names to a male/female/other convention

df['sex'].replace(to_replace=['Male', 'male', 'Male ','M','m','man','Male.','male 9:1 female, roughly','Male (cis)','Sex is male','Man',
                              'cis male','Malr','Dude',"I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
                              'mail', 'M|', 'Male/genderqueer','male ','Cis Male', 'Male (trans, FtM)','cisdude','cis man','MALE','Cis male'], value='MALE', inplace=True)

df['sex'].replace(to_replace=['Female', 'female', 'I identify as female.','female ','Female assigned at birth ','F', 'Woman', 'fm', 'f', 'Cis female ', 'Transitioned, M2F',
                                        'Female or Multi-Gender Femme', 'Female ', 'woman', 'female/woman','Cisgender Female','genderqueer woman','mtf','fem', 'Female (props for making this a freeform field, though)',
                                        ' Female','Cis-woman','Transgender woman'
                                        ],value='FEMALE', inplace=True)

df['sex'].replace(to_replace=['Bigender', 'non-binary',
       'Genderfluid (born female)', 'Other/Transfeminine', 'Androgynous', 'Other', 'nb masculine', 'none of your business',
       'genderqueer', 'Human', 'Genderfluid', 'Enby', 'Queer', 'Agender',
       'Fluid', 'Nonbinary', 'human', 'Unicorn', 'Genderqueer',
       'Genderflux demi-girl', 'female-bodied; no feelings about gender',
       'AFAB'], value='OTHER', inplace=True)

In [227]:
def show_value_counts(df):
    for column in df.columns:
        print(df[column].value_counts(), end='\n\n')

In [228]:
def show_no_missing_per_column(df):
    for column in df.columns:
        print(df[column].isna().sum())

In [229]:
def get_unique_conditions(cond_series):
    conditions = cond_series.unique()

    condition_unsep = set()
    for condition in conditions:
        condition_unsep.add(condition)

    condition_set = set()
    pipes = []

    for condition in condition_unsep:
        if '|' in condition:
            pipes.append(condition.split('|'))
        else:
            condition_set.add(condition)

    for cond_list in pipes:
        for cond in cond_list:
            condition_set.add(cond)
    
    return list(condition_set)

In [230]:
#manually added based on unique_diagnosed_conditions_by_prof - faster to discern the actually unqique ones by hand
actually_unique_by_prof = set(['Autism Spectrum Disorder', 'Mood disorder', 'PTSD', 'PDD-NOS', 'Addictive Disorder', 'ADHD', 'ADD',
                   'Anxiety Disorder', 'Burnout', 'Dissociative Disorder', 'Eating Disorder', 'Gender Dysphoria', 
                   'Gender Identity Disorder', 'Intimate Disorder', 'MCD', 'OCD', 'Personality Disorder', 'Psychotic Disorder',
                   'SAD', 'Stress Response Syndromes','Substance Use Disorder','Suicidal Ideation'])


In [231]:
columns_to_not_encode = ['is_self_employed', 'is_tech_org', 'has_prev_employers', 'sought_treatment', 'us_state_livesin', 'us_state_worksin','work_position', 'age']
to_encode = [col for col in df.columns if col not in columns_to_not_encode]

#majority from USA, UK, CD, GER, NED, AUS - dropping others for now
df['country_worksin'].value_counts()
df = df.loc[df['country_worksin'].isin(['United States of America', 'United Kingdom', 'Canada', 'Germany', 'Netherlands', 'Australia'])]

In [232]:
imputer = SimpleImputer(strategy='most_frequent')

In [233]:
imputed = pd.DataFrame(data =imputer.fit_transform(df), columns=df.columns)

encoded = pd.get_dummies(imputed, columns=to_encode, dtype=float)
encoded.head()
encoded.drop(['work_position'], axis='columns', inplace=True)

In [234]:
pca = PCA(n_components=2, random_state=67)
pca.fit(encoded)

pca.explained_variance_ratio_

array([0.84205996, 0.01377431])

In [235]:
get_unique_conditions(df['work_position'])

tech_roles = ['Back-end Developer', 'Front-end Developer','DevOps/SysAdmin', 'Dev Evangelist/Advocate']

joined = '|'.join(tech_roles)


In [236]:
tech_df = df[df['work_position'].str.contains(joined)]
get_unique_conditions(tech_df['work_position'])

['Sales',
 'Executive Leadership',
 'HR',
 'Dev Evangelist/Advocate',
 'Support',
 'Designer',
 'Other',
 'Back-end Developer',
 'Front-end Developer',
 'DevOps/SysAdmin',
 'One-person shop',
 'Supervisor/Team Lead']

In [237]:
def has_tech_role(row):
    
    tech_roles = ['Back-end Developer', 'Front-end Developer','DevOps/SysAdmin', 'Dev Evangelist/Advocate']
    
    for role in tech_roles:
        if role in row['work_position']:
            return 1
    
    return 0


df['has_tech_role'] = df.apply(lambda row: has_tech_role(row), axis=1)
df.drop('role_is_IT', axis='columns', inplace=True)

In [238]:
df[['empl_provides_mh_cov', 'has_medical_cov_incl_mh']]

,empl_provides_mh_cov,has_medical_cov_incl_mh
0,Not eligible for coverage / N/A,NaN
1,No,NaN
2,No,NaN
3,NaN,1.0
4,Yes,NaN
...,...,...
1427,Yes,NaN
1428,NaN,1.0
1430,Yes,NaN
1431,I don't know,NaN


In [239]:
#Most respondents with missing values for employer coverage
df.loc[pd.isna(df['empl_provides_mh_cov'])]['has_medical_cov_incl_mh'].value_counts()

has_medical_cov_incl_mh
1.0    142
0.0     67
Name: count, dtype: int64

In [240]:
df.loc[pd.isna(df['has_medical_cov_incl_mh'])]['empl_provides_mh_cov'].value_counts()

empl_provides_mh_cov
Yes                                512
I don't know                       275
No                                 155
Not eligible for coverage / N/A     60
Name: count, dtype: int64

In [241]:
df.loc[(df['has_medical_cov_incl_mh'] == 0) | (pd.isna(df['has_medical_cov_incl_mh']))]['empl_provides_mh_cov'].value_counts()

empl_provides_mh_cov
Yes                                512
I don't know                       275
No                                 155
Not eligible for coverage / N/A     60
Name: count, dtype: int64

In [242]:
df.loc[df['empl_provides_mh_cov'] == 'Yes']['has_medical_cov_incl_mh'].value_counts()

Series([], Name: count, dtype: int64)

In [243]:
df.loc[df['is_self_employed'] == 1]

,is_self_employed,employee_count_bracket,is_tech_org,empl_provides_mh_cov,knows_mh_cov_options,empl_discussed_mh,empl_offers_mh_rsrcs,anon_protec,if_askfor_mh_medical_leave_how_easy,discuss_mh_with_empl_wouldcause_neg_conseq,discuss_ph_with_empl_wouldcause_neg_conseq,comfy_discussing_mh_with_coworkers,comfy_discussing_mh_with_supervisors,empl_takes_mh_asseriously_as_ph,observed_neg_conseq_for_coworkers_openabout_mh_inworkspace,has_medical_cov_incl_mh,knowsof_mh_resources,if_diag_would_reveal_toclients/bn_contacts,if_reveal_diag_toclient_didthis_impact_neg,if_diag_would_reveal_tocoworkers/employees,if_reveal_diag_tocoworker_didthis_impact_neg,productivity_isaffected_by_mh,percentage_worktime_affected_by_mh,has_prev_employers,prev_empl_provided_mh_benefits,was_aware_of_prevemployers_mhcare_options,prev_empl_discussed_mh,prev_empl_provided_mh_rsrc,prev_anon_protec,prev_discuss_mh_with_empl_wouldcause_neg_conseq,prev_discuss_ph_with_empl_wouldcause_neg_conseq,prev_wouldhavebeen_willing_discuss_mh_coworkers,prev_wouldhavebeen_willing_discuss_mh_supervisors,prev_empl_takes_mh_asseriously_as_ph,prev_observed_neg_conseq_for_coworkers_openabout_mh_inworkspace,willingto_bringup_ph_interview,willingto_bringup_mh_interview,believes_beingident_as_mh_wouldhurt_career,thinks_coworkers_wouldviewthem_neg_if_mh,howwilling_share_mh_with_friendsfamily,observed_badly_handled_response_to_mh_inworkplace,observed_madethem_notwantto_talk_about_mh,has_family_history_mh,hashad_mh_inpast,HAS_MH,diagnoses_notprof,conditions_suspected,hasbeen_diag_byprof,diagnoses_prof,sought_treatment,mh_interferes_w/work_effective_trt,mh_interferes_w/work_ineffective_trt,age,sex,country_worksin,work_position,is_remote_working,has_tech_role
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Yes, I know several","Sometimes, if it comes up",I'm not sure,"Sometimes, if it comes up",I'm not sure,Yes,1-25%,1,Some did,N/A (not currently aware),None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Neutral,No,NaN,No,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Sometimes,Sometimes,43,MALE,United Kingdom,Supervisor/Team Lead,Sometimes,0
9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,I know some,"No, because it doesn't matter",NaN,"Sometimes, if it comes up",No,Yes,1-25%,1,Some did,I was aware of some,None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Yes,Maybe,Maybe,"No, I don't think they would",Very open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Rarely,Often,30,MALE,United States of America,One-person shop|Front-end Developer|Back-end D...,Always,1
24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,I know some,Not applicable to me,NaN,Not applicable to me,Not applicable to me,Not applicable to me,NaN,1,I don't know,"No, I only became aware later",None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,Some of them,Maybe,Maybe,Maybe,"No, I don't think they would",Not applicable to me (I do not have a mental i...,NaN,NaN,No,No,No,NaN,NaN,No,NaN,0,Not applicable to me,Not applicable to me,38,MALE,United States of America,Supervisor/Team Lead,Sometimes,0
40,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,I know some,"No, because it doesn't matter",Yes,"Sometimes, if it comes up",I'm not sure,Yes,26-50%,1,"Yes, they all did","No, I only became aware later",None did,Some did,"Yes, always",Some of them,Some of them,"Yes, at all of my previous employers",Some of my previous employers,None did,Some of them,Maybe,No,"Yes, I think it would","No, I don't think they would",Somewha

In [244]:
"""
#Changing the nan value for self employed to "N/A" (Not applicable), because self-employed ppl can't answer the employee count question
df.loc[df['is_self_employed'] == 1, 'employee_count_bracket'] = df.loc[df['is_self_employed'] == 1, 'employee_count_bracket'].fillna("N/A")

#same for employer provided insurance
df.loc[df['is_self_employed'] == 1, 'empl_provides_mh_cov']   = df.loc[df['is_self_employed'] == 1, 'empl_provides_mh_cov'].fillna("N/A")

#same for whether the company is tech
df.loc[df['is_self_employed'] == 1, 'is_tech_org']            = df.loc[df['is_self_employed'] == 1, 'is_tech_org'].fillna("N/A")

"""


'\n#Changing the nan value for self employed to "N/A" (Not applicable), because self-employed ppl can\'t answer the employee count question\ndf.loc[df[\'is_self_employed\'] == 1, \'employee_count_bracket\'] = df.loc[df[\'is_self_employed\'] == 1, \'employee_count_bracket\'].fillna("N/A")\n\n#same for employer provided insurance\ndf.loc[df[\'is_self_employed\'] == 1, \'empl_provides_mh_cov\']   = df.loc[df[\'is_self_employed\'] == 1, \'empl_provides_mh_cov\'].fillna("N/A")\n\n#same for whether the company is tech\ndf.loc[df[\'is_self_employed\'] == 1, \'is_tech_org\']            = df.loc[df[\'is_self_employed\'] == 1, \'is_tech_org\'].fillna("N/A")\n\n'

In [245]:
df.loc[df['is_self_employed'] == 1, ['employee_count_bracket', 'empl_provides_mh_cov']] = df.loc[df['is_self_employed'] == 1, ['employee_count_bracket', 'empl_provides_mh_cov']].fillna("N/A")

In [246]:
df.loc[df['is_self_employed'] == 1, ['employee_count_bracket', 'empl_provides_mh_cov']]

,employee_count_bracket,empl_provides_mh_cov
3,N/A,N/A
9,N/A,N/A
24,N/A,N/A
40,N/A,N/A
46,N/A,N/A
...,...,...
1382,N/A,N/A
1404,N/A,N/A
1408,N/A,N/A
1420,N/A,N/A


In [ ]:
excluded_for_self_employed = ['employee_count_bracket', 'empl_provides_mh_cov', 'is_tech_org', 'empl_discussed_mh', 'empl_offers_mh_rsrcs']

In [247]:
df.loc[df['is_self_employed'] == 1, 'empl_discussed_mh']

3       NaN
9       NaN
24      NaN
40      NaN
46      NaN
       ... 
1382    NaN
1404    NaN
1408    NaN
1420    NaN
1428    NaN
Name: empl_discussed_mh, Length: 209, dtype: object

In [248]:
df.loc[df['is_self_employed'] == 1]

,is_self_employed,employee_count_bracket,is_tech_org,empl_provides_mh_cov,knows_mh_cov_options,empl_discussed_mh,empl_offers_mh_rsrcs,anon_protec,if_askfor_mh_medical_leave_how_easy,discuss_mh_with_empl_wouldcause_neg_conseq,discuss_ph_with_empl_wouldcause_neg_conseq,comfy_discussing_mh_with_coworkers,comfy_discussing_mh_with_supervisors,empl_takes_mh_asseriously_as_ph,observed_neg_conseq_for_coworkers_openabout_mh_inworkspace,has_medical_cov_incl_mh,knowsof_mh_resources,if_diag_would_reveal_toclients/bn_contacts,if_reveal_diag_toclient_didthis_impact_neg,if_diag_would_reveal_tocoworkers/employees,if_reveal_diag_tocoworker_didthis_impact_neg,productivity_isaffected_by_mh,percentage_worktime_affected_by_mh,has_prev_employers,prev_empl_provided_mh_benefits,was_aware_of_prevemployers_mhcare_options,prev_empl_discussed_mh,prev_empl_provided_mh_rsrc,prev_anon_protec,prev_discuss_mh_with_empl_wouldcause_neg_conseq,prev_discuss_ph_with_empl_wouldcause_neg_conseq,prev_wouldhavebeen_willing_discuss_mh_coworkers,prev_wouldhavebeen_willing_discuss_mh_supervisors,prev_empl_takes_mh_asseriously_as_ph,prev_observed_neg_conseq_for_coworkers_openabout_mh_inworkspace,willingto_bringup_ph_interview,willingto_bringup_mh_interview,believes_beingident_as_mh_wouldhurt_career,thinks_coworkers_wouldviewthem_neg_if_mh,howwilling_share_mh_with_friendsfamily,observed_badly_handled_response_to_mh_inworkplace,observed_madethem_notwantto_talk_about_mh,has_family_history_mh,hashad_mh_inpast,HAS_MH,diagnoses_notprof,conditions_suspected,hasbeen_diag_byprof,diagnoses_prof,sought_treatment,mh_interferes_w/work_effective_trt,mh_interferes_w/work_ineffective_trt,age,sex,country_worksin,work_position,is_remote_working,has_tech_role
3,1,N/A,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Yes, I know several","Sometimes, if it comes up",I'm not sure,"Sometimes, if it comes up",I'm not sure,Yes,1-25%,1,Some did,N/A (not currently aware),None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Neutral,No,NaN,No,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Sometimes,Sometimes,43,MALE,United Kingdom,Supervisor/Team Lead,Sometimes,0
9,1,N/A,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,I know some,"No, because it doesn't matter",NaN,"Sometimes, if it comes up",No,Yes,1-25%,1,Some did,I was aware of some,None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Yes,Maybe,Maybe,"No, I don't think they would",Very open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Rarely,Often,30,MALE,United States of America,One-person shop|Front-end Developer|Back-end D...,Always,1
24,1,N/A,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,I know some,Not applicable to me,NaN,Not applicable to me,Not applicable to me,Not applicable to me,NaN,1,I don't know,"No, I only became aware later",None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,Some of them,Maybe,Maybe,Maybe,"No, I don't think they would",Not applicable to me (I do not have a mental i...,NaN,NaN,No,No,No,NaN,NaN,No,NaN,0,Not applicable to me,Not applicable to me,38,MALE,United States of America,Supervisor/Team Lead,Sometimes,0
40,1,N/A,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,I know some,"No, because it doesn't matter",Yes,"Sometimes, if it comes up",I'm not sure,Yes,26-50%,1,"Yes, they all did","No, I only became aware later",None did,Some did,"Yes, always",Some of them,Some of them,"Yes, at all of my previous employers",Some of my previous employers,None did,Some of them,Maybe,No,"Yes, I think it would","No, I don't think they would",Somewha